In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import Cursor
import ipywidgets as widgets
from matplotlib.widgets import RadioButtons
from numpy.fft import fft, fftshift
from scipy.optimize import curve_fit, minimize
import math
import time
import originpro as op
%matplotlib widget
import os
import cmath

import sys
def origin_shutdown_exception_hook(exctype, value, traceback):
    '''Ensures Origin gets shut down if an uncaught exception'''
    op.exit()
    sys.__excepthook__(exctype, value, traceback)
if op and op.oext:
    sys.excepthook = origin_shutdown_exception_hook

In [2]:
intpoints = 2048

#thickness in mm
d                = 1.5                   #mm
ref_filename     = '120121.004'    #Reference Filename
sample_filename  = '120121.005'    #Sample filename
cutoff_frequencyend = 2.0                  #cutoff frequency for optimization in THz 
cutoff_frequencystart = 0.4
#path =    #Filepath to save originfile

#FP parameters
#1 is medium before sample
#2 is sample medium
#3 is medium after sample
n1 = np.full((int(intpoints/2),),1) + 1j*np.full((int(intpoints/2),),0)
n2 = np.full((int(intpoints/2),),2) + 1j*np.full((int(intpoints/2),),0.02)
n3 = np.full((int(intpoints/2),),1) + 1j*np.full((int(intpoints/2),),0)

#FP = 1/(1-A*B*Exp(C))
#A = (n2-n1)/(n2+n1)
#B = (n2-n3)/(n2+n3)
#C = -2*j*n2*w*L/c
#FP = 1/(())

#Uncomment next line for thick sample
#FP = 1


#Constants
c = 299792458 #m/s (Speed of light)
alpha_constant = 419.16900439033634     #(4*np.pi/299792458)*1e12(THz)*1e-2(m to cm)

In [3]:
#FP = 1/(1-A*B*Exp(C))
#A = (n2-n1)/(n2+n1)
#B = (n2-n3)/(n2+n3)
#C = -2*j*n2*w*L/c

In [ ]:
FP = 1/(1-((n2-n1)/(n2+n1))*((n2-n3)/(n2+n3))*np.exp(-1j*(2*n2[:int(intpoints/2)])/const))

In [ ]:
plt.plot(np.unwrap(np.angle(FP)))

In [ ]:
plt.close()

In [ ]:
plt.plot(fft_freq,np.absolute(FP))

In [4]:
def readfile(filename):
    import pandas as pd
    import numpy as np
    
    df = pd.read_csv(filename,skiprows=12,sep='\s+').iloc[:-1,:5]
    df=pd.DataFrame(np.array(df, dtype=float))
    index=["Distance (um)", "Delay (ps)", "Signal (V)", "Phase (deg)", "Rcos (theta)"]
    df=df.set_axis(index, axis=1, inplace=False).reset_index(drop=True)
    return df

class SnaptoCursor(object):
    def __init__(self, ax, x, y):
        self.ax = ax
        self.lx = ax.axhline(color='k',lw=0.5)  # the horiz line
        self.ly = ax.axvline(color='k')         # the vert line
        self.x = x
        self.y = y

        # text location in axes coords
        self.txt = ax.text(0.7, 0.9, '', transform=ax.transAxes)

    def mouse_buttonpress(self, event):
        if not event.inaxes:
            return
        x, y = event.xdata, event.ydata
        indx = min(np.searchsorted(self.x, x), len(self.x) - 1)
        
        index_temp.append(indx)
        
        x = self.x[indx]
        y = self.y[indx]
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)
        
        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        print('x=%1.2f, y=%1.2f' % (x, y))

        self.ax.figure.canvas.draw()

#Start button
def startclicked(arg):
    global timestartindex
    timestartindex=index_temp[-1]
    print('Start Index: ',timestartindex)


#End Button
def endclicked(arg):
    global timeendindex
    timeendindex=index_temp[-1]
    print('End Index  : ',timeendindex)
    
    
    
##
#Phase Correction Reference
##
class SnaptoCursor1(object):
    def __init__(self, ax, x, y):
        self.ax = ax
        self.lx = ax.axhline(color='k',lw=0.5)  # the horiz line
        self.ly = ax.axvline(color='k')         # the vert line
        self.x = x
        self.y = y
        
        # text location in axes coords
        self.txt = ax.text(0.7, 0.1, '', transform=ax.transAxes)

    def snap(self, event):
        if not event.inaxes:
            return
        print(index_temp)
        x, y = event.xdata, event.ydata
        indx = min(np.searchsorted(self.x, x), len(self.x) - 1)
        index_temp.append(indx)
    
        x = self.x[indx]
        y = self.y[indx]
        
        # update the line positions
        self.lx.set_ydata(y)
        self.ly.set_xdata(x)
        self.txt.set_text('x=%1.2f, y=%1.2f' % (x, y))
        self.ax.figure.canvas.draw()

#Reference
def add2pi_ref(arg):
    global add2pi
    add2pi=index_temp[-1]
    new_y = add(add2pi, temp)
    plot.set_ydata(new_y)
    print('Add Index: ',add2pi)

def sub2pi_ref(arg):
    global sub2pi
    sub2pi=index_temp[-1]
    new_y = sub(sub2pi,temp)
    plot.set_ydata(new_y)
    print('Sub Index: ',sub2pi)

#Sample
def add2pi_sample(arg):
    global add2pi
    add2pi=index_temp[-1]
    new_y = add(add2pi, temp)
    plot.set_ydata(new_y)
    print('Add Index: ',add2pi)

def sub2pi_sample(arg):
    global sub2pi
    sub2pi=index_temp[-1]
    new_y = sub(sub2pi,temp)
    plot.set_ydata(new_y)
    print('Sub Index: ',sub2pi)

#Phase Difference
def sub2pi_phasediff(arg):
    global sub2pi
    sub2pi=index_temp[-1]
    new_y = sub(sub2pi,temp)
    plot.set_ydata(new_y)
    print('Sub Index: ',sub2pi)
    
def add2pi_phasediff(arg):
    global add2pi
    add2pi=index_temp[-1]
    new_y = add(add2pi, temp)
    plot.set_ydata(new_y)
    print('Add Index: ',add2pi)

def add(index, temp):
    new_y = temp;
    new_y[index:] = new_y[index:]+2*np.pi
    temp = new_y
    return new_y

def sub(index, temp):
    new_y = temp;
    new_y[index:] = new_y[index:]-2*np.pi
    temp = new_y
    return new_y
  
def linearfit(x, m, c):
    return m*x + c

def find_nearest_index(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx

'''def n_k_2_phase_trans(n, k, d, freq):
    #Optical Parameter Extraction "THz TDS" -Caihong Zhang 31-03-2008 Thesis for Optically thick samples
    #trans(w) = (4*n/(n+1)^2)*exp(-kwd/c)
    #phase(w) = (n-1)wd/c
    
    wd_c = 20.958450219516816*freq[cutoff_indexstart:cutoff_indexend]*d      #wd_c = (2*np.pi*freq*d)/c, but 2*np.pi*(THz)*(mm)/c value is 20.95845021951681

    calculated_transmission = (4*n[cutoff_indexstart:cutoff_indexend]/(n[cutoff_indexstart:cutoff_indexend]+1)**2)*np.exp(-k[cutoff_indexstart:cutoff_indexend]*wd_c)
    calculated_phase = (n[cutoff_indexstart:cutoff_indexend]-1)*wd_c
    
    return calculated_transmission, calculated_phase'''

#def errorfunc(trans_meas, phase_meas, freq, thickness):
'''def errorfunc(thickness):
    #trans and phase is transmission and phase calculated from n and k
    #trans_meas and phase_meas is transmission and phase measured experimentally
    #phase and phase_meas is in radian
    #Refer "A reliable method for extraction of Material parameters in THz TDS, IEEE(1996),Jean-Louis Coutaz"
    
    trans, phase = n_k_2_phase_trans(n, k, thickness, freq)
    
    del_T = np.log(abs(trans/trans_meas[cutoff_indexstart:cutoff_indexend]))
    del_P = phase - phase_meas[cutoff_indexstart:cutoff_indexend]
    
    error = np.median(del_T**2 + del_P**2)
    #return del_T**2 + del_P**2
    return error ''' 

def plot_delay(xr, yr, xs, ys, xlabel, ylabel, title):
    plt.close()
    plt.close()
    
    fig, ax = plt.subplots()
    ax.set_ylim((-float(yr.max())*1.2,float(yr.max())*1.2))
    #ax.set_xlim((float(xr.min()),float(xr.max())))
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.plot(xr, yr, '-',label='Reference',markersize=5)
    ax.plot(xs, ys, '-',label='Sample',markersize=5)
    ax.legend(loc='upper right')
    ax.grid(which = 'major',color='lightgray', linestyle='-', linewidth=1, alpha = 0.4)
    plt.show()
    
def plot_amplitude(fft_freq, fft_ampref, fft_ampsample, xmin, xmax, ymin, ymax):
    plt.close()
    plt.close()
    
    plt.plot(fft_freq, fft_ampref, label='Reference',markersize=5)
    plt.plot(fft_freq, fft_ampsample, label='Sample',markersize=5)
    plt.yscale('log')
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Amplitude (a.u.)')
    plt.title('FFT Amplitude')
    plt.legend(loc='upper right')
    plt.show()
    
def plot_phase(fft_freq, fft_ampref, fft_ampsample, xmin, xmax, ymin, ymax):
    plt.close()
    plt.close()
    
    plt.plot(fft_freq, fft_phaseref, label='Reference',markersize=5, marker='o')
    plt.plot(fft_freq, fft_phasesample, label='Sample',markersize=5, marker='o')
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Phase (rad.)')
    plt.title('FFT Phase')
    plt.legend(loc='upper right')
    plt.show()
    
def plot_transmission(X, Y, xmin, xmax):
    plt.close()
    plt.close()
    
    plt.plot(X, Y, label='Transmission',markersize=5)
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(0,1)
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Transmission')
    plt.title('Transmission')
    plt.legend(loc='lower right')
    plt.show()
    
def plot_phasediff(X, Y, xmin, xmax, ymin, ymax):
    plt.close()
    plt.close()
    
    plt.plot(X, Y, label='Tranmission',markersize=5, marker='o')
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Phase Difference (rad)')
    plt.title('Phase Difference')
    plt.legend(loc='upper right')
    plt.show()
    
def plot_n(X, Y, xmin, xmax, ymin, ymax):
    plt.close()    
    plt.plot(X, Y, label='n',markersize=5)
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Refractive Index')
    plt.title('Refractive Index')
    plt.legend(loc='upper right')
    plt.show()  
    
def plot_k(X, Y, xmin, xmax, ymin, ymax):
    plt.close()    
    plt.plot(X, Y, label='k',markersize=5)
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.yscale('log')
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Extinction Coefficient')
    plt.title('Extinction Coefficient')
    plt.legend(loc='upper right')
    plt.show()
    
def plot_alpha(X, Y, xmin, xmax, ymin, ymax):
    plt.close()    
    plt.plot(X, Y, label='α',markersize=5)
    plt.xlim(xmin,xmax)   #0 to 3THz
    plt.ylim(ymin,ymax)
    plt.yscale('log')
    plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
    plt.grid(which = 'minor',color='lightgray', linestyle='--', linewidth=1, alpha = 0.4)
    plt.xlabel('Frequency (THz)')
    plt.ylabel('Absorption coefficient (cm-1)')        
    plt.title('Absorption Coefficient')
    plt.legend(loc='upper right')
    plt.show()

In [5]:
#test=readfile('test.txt')
reference=readfile(ref_filename)
sample=readfile(sample_filename)

#print(test.iloc[:,[1,4]])
#timeplot(test,'Reference Signal')

In [6]:
index_temp = []   #Temporary Index values of start and stop button

#Reference
#Reads Reference delay(ps) and Rcos(theta) as numpy array and stores in a variable
xr=np.array(reference.iloc[:,1])   #delay axis
yr=np.array(reference.iloc[:,4])   #Rcos(theta) axis

#Sample
#Reads Sample delay(ps) and Rcos(theta) as numpy array and stores in a variable
xs=np.array(sample.iloc[:,1])   #delay axis
ys=np.array(sample.iloc[:,4])   #Rcos(theta) axis


#Plots Reference and Sample delay plot
fig, ax = plt.subplots()
ax.set_ylim((-float(yr.max())*1.2,float(yr.max())*1.2))
ax.set_xlim((float(xr.min()),float(xr.max())))
ax.set_xlabel('Delay (ps)')
ax.set_ylabel('Amplitude (V)')
ax.set_title('Signal-Reference Time Domain Plot')
ax.plot(xr, yr, '-',label='Reference',markersize=5)
ax.plot(xs, ys, '-',label='Sample',markersize=5)
plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)

#Delay plot Buttons
button_start = widgets.Button(description = 'Start')   
button_start.on_click(startclicked)
display(button_start)

button_end = widgets.Button(description = 'End')   
button_end.on_click(endclicked)
display(button_end)

#snapping cursor to Reference
snap_cursor = SnaptoCursor(ax, xr, yr)
fig.canvas.mpl_connect('button_press_event', snap_cursor.mouse_buttonpress)
plt.legend(loc='upper left')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Start', style=ButtonStyle())

Button(description='End', style=ButtonStyle())

Start Index:  35
End Index  :  1418


In [7]:
#Rectangular Windowning
trim_delayref = reference.iloc[timestartindex:timeendindex+1,[1,4]]
trim_delaysample = sample.iloc[timestartindex:timeendindex+1,[1,4]]

#Interpolated Delay axis generator
delaystart = trim_delayref.to_numpy()[0][0]
delayend   = trim_delayref.to_numpy()[-1][0]
#delaytotal = (delayend-delaystart)/149.89623
delaytotal = (delayend-delaystart)
delaydelta = delaytotal/(intpoints-1)
int_delay  = np.arange(0,intpoints)*delaydelta+delaystart
#int_delay  = np.arange(delaystart/149.89623,(delayend+0.001)/149.89623,delaydelta)

#print(int_delay)
#Interpolation of sample and reference
int_delayref    = np.interp(int_delay,trim_delayref.to_numpy()[:,0],trim_delayref.to_numpy()[:,1]).reshape(intpoints)
int_delaysample = np.interp(int_delay,trim_delaysample.to_numpy()[:,0],trim_delaysample.to_numpy()[:,1]).reshape(intpoints)

#Interpolated delay plotting
plot_delay(int_delay,int_delayref,
     int_delay,int_delaysample,
     'Delay (ps)',
     'Amplitude (V)',
     'Signal-Reference Time Domain Plot (Windowing)')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [8]:
#Fast Fourier Transform using scipy.fft
#FFT Frequency axis Generator
fft_freq      = np.arange(0,intpoints/2,1)/delaytotal                                       #Frequency Axis 

#FFT Sample
fft_ampref    = abs(fftshift(fft(int_delayref)))[int(intpoints/2):]                         #Amplitude Axis
fft_phaseref  = np.unwrap(np.angle((fftshift(fft(int_delayref)))[int(intpoints/2):]))       #Phase axis

#FFT Reference
fft_ampsample    = abs(fftshift(fft(int_delaysample)))[int(intpoints/2):]                   #Amplitude Axis
fft_phasesample  = np.unwrap(np.angle((fftshift(fft(int_delaysample)))[int(intpoints/2):])) #Phase axis

#Plot Amplitude
plot_amplitude(fft_freq, fft_ampref, fft_ampsample, 0, 3, 1e-3, 10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [9]:
#Plot Phase
plot_phase(fft_freq, fft_phaseref, fft_phasesample, 0, 3, -100, 300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
#Reference Phase Correction
temp=[]
temp = fft_phaseref

#Plots Reference and Sample delay plot
plt.close()
plt.close()
fig, ax = plt.subplots()
#ax.set_ylim(-200,4)
ax.set_xlim(0,3)
ax.set_xlabel('Frequency (THz)')
ax.set_ylabel('Phase (rad)')
ax.set_title('Phase (Reference)')
plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
plot, = ax.plot(fft_freq, fft_phaseref, '-',label='Reference',marker='o')
plt.legend(loc='upper right')
#ax.plot(xs, ys, '-',label='Sample',markersize=5)'''


#Delay plot Buttons
button_add = widgets.Button(description = 'Add 2*pi')   
button_add.on_click(add2pi_ref)
display(button_add)

button_sub = widgets.Button(description = 'Sub 2*pi')   
button_sub.on_click(sub2pi_ref)
display(button_sub)

#snapping cursor to Reference
snap_cursor1 = SnaptoCursor1(ax, fft_freq, fft_phaseref)
fig.canvas.mpl_connect('button_press_event', snap_cursor1.snap)
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Add 2*pi', style=ButtonStyle())

Button(description='Sub 2*pi', style=ButtonStyle())

In [11]:
phaseref_corrected = temp

In [12]:
#Sample Phase Correction
temp = fft_phasesample

#Plots Reference and Sample delay plot
plt.close()
plt.close()
fig, ax = plt.subplots()
#ax.set_ylim(-200,4)
ax.set_xlim(0,3)
ax.set_xlabel('Frequency (THz)')
ax.set_ylabel('Phase (rad)')
ax.set_title('Phase (Sample)')
plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
plot, = ax.plot(fft_freq, fft_phasesample, '-',label='Sample',marker='o')
plt.legend(loc='upper right')
#ax.plot(xs, ys, '-',label='Sample',markersize=5)'''


#Delay plot Buttons
button_add = widgets.Button(description = 'Add 2*pi')   
button_add.on_click(add2pi_sample)
display(button_add)

button_sub = widgets.Button(description = 'Sub 2*pi')   
button_sub.on_click(sub2pi_sample)
display(button_sub)

#snapping cursor to Reference
snap_cursor1 = SnaptoCursor1(ax, fft_freq, fft_phasesample)
fig.canvas.mpl_connect('button_press_event', snap_cursor1.snap)
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Add 2*pi', style=ButtonStyle())

Button(description='Sub 2*pi', style=ButtonStyle())

In [13]:
phasesample_corrected = temp

In [14]:
#phasediff = fft_phasesample -fft_phaseref
phasediff = (phasesample_corrected - phaseref_corrected)
#plot_phasediff(fft_freq, phasediff, 0, 2, -10,100)

#Sample Phase Correction
temp = phasediff

#Plots Reference and Sample delay plot
plt.close()
plt.close()
fig, ax = plt.subplots()
#ax.set_ylim(-200,4)
ax.set_xlim(0,3)
ax.set_xlabel('Frequency (THz)')
ax.set_ylabel('Phase (rad)')
ax.set_title('Phase Difference')
plt.grid(which = 'major',color='black', linestyle='-', linewidth=0.5, alpha = 0.4)
plot, = ax.plot(fft_freq, phasediff, '-',label='Sample',marker='o')
plt.legend(loc='upper right')
#ax.plot(xs, ys, '-',label='Sample',markersize=5)'''


#Delay plot Buttons
button_add = widgets.Button(description = 'Add 2*pi')   
button_add.on_click(add2pi_phasediff)
display(button_add)

button_sub = widgets.Button(description = 'Sub 2*pi')   
button_sub.on_click(sub2pi_phasediff)
display(button_sub)

button_start = widgets.Button(description = 'Start')   
button_start.on_click(startclicked)
display(button_start)

button_end = widgets.Button(description = 'End')   
button_end.on_click(endclicked)
display(button_end)

#snapping cursor to Reference
snap_cursor1 = SnaptoCursor1(ax, fft_freq, phasediff)
fig.canvas.mpl_connect('button_press_event', snap_cursor1.snap)
plt.legend(loc='upper right')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Button(description='Add 2*pi', style=ButtonStyle())

Button(description='Sub 2*pi', style=ButtonStyle())

Button(description='Start', style=ButtonStyle())

Button(description='End', style=ButtonStyle())

Start Index:  22
End Index  :  81


In [17]:
phasediff_corrected = temp

#Next four lines finds the y intercept of the phase difference
trim_fft_freq = fft_freq[timestartindex:timeendindex+1]
trim_phasediff_corrected = phasediff_corrected[timestartindex:timeendindex+1]
popt, pconv = curve_fit(linearfit, trim_fft_freq, trim_phasediff_corrected)
y_intercept = popt[1]

In [18]:
transmission = fft_ampsample/fft_ampref
plot_transmission(fft_freq, transmission, 0, 3)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [19]:
#C/wd
const = []
const.append(1)
const[1:] = (c*1000)/(2*np.pi*fft_freq[1:]*d*1e12)     #c/omega*d   freq in THz & d in mm
const = np.array(const)

#Refractive Index
n = abs(1-(np.multiply(phasediff_corrected+y_intercept,const)))
plot_n(fft_freq, n, 0.4, 2.8, 0, 10)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
#Extinction Coefficient
k = abs(const*np.log((4*n)/(transmission*(n+1)**2)))
plot_k(fft_freq, k, 0, 2.8, 0.001, 1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
#alpha Absorption Coefficient
alpha = alpha_constant*k*fft_freq
plot_alpha(fft_freq, alpha, 0.4, 2.8, 0.1, 100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
#def n_k_2_phase_trans(n, k, d, freq):
def n_k_2_phase_trans(n_1, n_2, n_3, freq): 
    #Optical Parameter Extraction "THz TDS" -Caihong Zhang 31-03-2008 Thesis for Optically thick samples
    #trans(w) = (4*n/(n+1)^2)*exp(-kwd/c)
    #phase(w) = (n-1)wd/c
    
    wd_c = 20.958450219516816*freq[cutoff_indexstart:cutoff_indexend]*d      #wd_c = (2*np.pi*freq*d)/c, but 2*np.pi*(THz)*(mm)/c value is 20.95845021951681
    n_1 = n_1[cutoff_indexstart:cutoff_indexend]
    n_2 = n_2[cutoff_indexstart:cutoff_indexend]
    n_3 = n_3[cutoff_indexstart:cutoff_indexend]
    #calculated_transmission = (4*n[cutoff_indexstart:cutoff_indexend]/(n[cutoff_indexstart:cutoff_indexend]+1)**2)*np.exp(-k[cutoff_indexstart:cutoff_indexend]*wd_c)
    #calculated_phase = (n[cutoff_indexstart:cutoff_indexend]-1)*wd_c
    
    calculated_transmission = (2*n_2*(n_1+n_3)/((n_2+n_1)*(n_2+n_3)))*np.exp(np.imag(n_2)*wd_c)
    calculated_phase = (n_2-1)*wd_c
    
    return calculated_transmission, calculated_phase


def errorfunc(n_2):
    #trans and phase is transmission and phase calculated from n and k
    #trans_meas and phase_meas is transmission and phase measured experimentally
    #phase and phase_meas is in radian
    #Refer "A reliable method for extraction of Material parameters in THz TDS, IEEE(1996),Jean-Louis Coutaz"
    n_real = n_2[:1024]
    n_img = n_2[1024:]
    
    trans, phase = n_k_2_phase_trans(n1, n_real-1j*n_img, n3, freq)
    
    #del_T = np.log(abs(trans/trans_meas[cutoff_indexstart:cutoff_indexend]))
    #del_P = phase - phase_meas[cutoff_indexstart:cutoff_indexend]
    #print(newtrans_meas)
    
    del_T = np.log(abs(trans/newtrans_meas[cutoff_indexstart:cutoff_indexend]))
    del_P = phase - newphase_meas[cutoff_indexstart:cutoff_indexend]
    
    error = np.median(del_T**2 + del_P**2)
    #return del_T**2 + del_P**2
    return error

In [39]:
cutoff_indexend = find_nearest_index(fft_freq, cutoff_frequencyend)
cutoff_indexstart = find_nearest_index(fft_freq, cutoff_frequencystart)

#Below four lines are the global variables, I have used this names in the function
thickness = d
trans_meas = transmission
phase_meas = abs(phasediff_corrected)
freq = fft_freq

n_2 = np.array((n,k))

#FP parameter
FP = 1/(1-((n2-n1)/(n2+n1))*((n2-n3)/(n2+n3))*np.exp(-1j*(2*n2[:int(intpoints/2)])/const))

newtrans_meas = trans_meas/np.absolute(FP)
newphase_meas = phase_meas + np.unwrap(np.angle(FP))

#n_2 = np.array((n,k))

'''
###
Nelder mead specific Optimization options
https://docs.scipy.org/doc/scipy/reference/optimize.minimize-neldermead.html#optimize-minimize-neldermead
####
General Optimization options 
https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html#scipy.optimize.minimize
'''

result = minimize(errorfunc, n_2, method='nelder-mead', options = {'maxiter': 1, 'maxiter': 1})
#result = minimize(errorfunc, n2, method='nelder-mead')
#print('Optimized Thickness :', np.ndarray.item(result['x']), 'mm')
print(result)

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

...

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01 4.20218374e-01 ... 1.21832077e+07
 1.17646100e+07 1.39150785e+08]
[1.63628251e-01 3.53001752e-01

In [36]:
FP

array([ 9.21907422e-01+7.31291583e-02j,  9.53054359e-01-9.79050807e-02j,
        9.15366048e-01+6.38715734e-02j, ...,
        1.79071970e-08+1.27494286e-08j, -1.80256380e-08+1.18271659e-08j,
       -4.39261004e-09-2.06833670e-08j])

In [26]:
a[:1024]-n

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
'''#Origin Saving
raw      = op.new_sheet(lname='Measured Data')
analysis = op.new_sheet(lname=('analysis_'+str(d)+'mm'))

#Reference Data
raw.from_list(0, reference.to_numpy()[:,0], 'Distance',    units = 'um',  comments='Reference',axis=None)
raw.from_list(1, reference.to_numpy()[:,1], 'Delay',       units = 'ps',  comments='Reference',axis=None)
raw.from_list(2, reference.to_numpy()[:,2], 'Signal',      units = 'V',   comments='Reference',axis=None)
raw.from_list(3, reference.to_numpy()[:,3], 'Phase',       units = 'deg', comments='Reference',axis=None)
raw.from_list(4, reference.to_numpy()[:,4], 'Rcos(theta)', units = '',    comments='Reference',axis=None)

#Sample Data
raw.from_list(6,  sample.to_numpy()[:,0], 'Distance',    units = 'um',  comments='Sample',axis=None)
raw.from_list(7,  sample.to_numpy()[:,1], 'Delay',       units = 'ps',  comments='Sample',axis=None)
raw.from_list(8,  sample.to_numpy()[:,2], 'Signal',      units = 'V',   comments='Sample',axis=None)
raw.from_list(9,  sample.to_numpy()[:,3], 'Phase',       units = 'deg', comments='Sample',axis=None)
raw.from_list(10, sample.to_numpy()[:,4], 'Rcos(theta)', units = '',    comments='Sample',axis=None)

#Analysis Data
analysis.from_list(0, int_delay,       'IntDelay',       units = 'ps', comments='',         axis='X')
analysis.from_list(1, int_delayref,    'IntRcos(theta)', units = '',   comments='Reference',axis='Y')
analysis.from_list(2, int_delaysample, 'IntRcos(theta)', units = '',   comments='Sample',   axis='Y')

analysis.from_list(4, fft_freq,     'Frequency', units = 'THz',  comments='',         axis='X')
analysis.from_list(5, fft_ampref,   'Amplitude', units = 'a.u.', comments='Reference',axis='Y')
analysis.from_list(6, fft_ampsample,'Amplitude', units = 'a.u.', comments='Sample',   axis='Y')

analysis.from_list(7, fft_phaseref,   'Phase',     units = 'rad', comments='Reference',axis='Y')
analysis.from_list(8, fft_phasesample,'Phase',     units = 'rad', comments='Sample',   axis='Y')

analysis.from_list(7, phaseref_corrected,   'Corrected Phase',     units = 'rad', comments='Reference',axis='Y')
analysis.from_list(8, phasesample_corrected,'Corrected Phase',     units = 'rad', comments='Sample',   axis='Y')

analysis.from_list(9,  transmission,       'Transmission',               units = '',    comments='', axis='Y')
analysis.from_list(10, phasediff,          'Phase Difference',           units = 'rad', comments='', axis='Y')
analysis.from_list(11, phasediff_corrected,'Corrected Phase Difference', units = 'rad', comments='', axis='Y')

analysis.from_list(12, n,    'n', units = '',    comments='Refractive Index',       axis='Y')
analysis.from_list(13, k,    'k', units = '',    comments='Extinction Coefficient', axis='Y')
analysis.from_list(14, alpha,'α', units = '1/cm',comments='Absoption coefficient',  axis='Y')

op.save(os.path.normpath(os.getcwd()) + r'\result.opju')
#op.save(os.path.normpath(os.path.normpath(path)) + r'\result.opju')
op.exit()'''

In [ ]:
#Optimised n
#C/wd
const = []
const.append(1)
const[1:] = (c*1000)/(2*np.pi*fft_freq[1:]*np.ndarray.item(result['x'])*1e12)     #c/omega*d
const = np.array(const)

#Refractive Index
n_d_opt = abs(1-(np.multiply(phasediff_corrected+y_intercept,const)))
plot_n(fft_freq, n_d_opt, 0.4, 2.8, 2, 4)

In [ ]:
#Extinction Coefficient
k_d_opt = abs(const*np.log((4*n_d_opt)/(transmission*(n_d_opt+1)**2)))
plot_k(fft_freq, k_d_opt, 0, 2.8, 0.001, 1)